In [3]:
import math
import sys

import numpy as np

sys.path.append('../')
sys.path.append('../../')
from run_utils import get_env_dataset, run_env_experiment
from run_utils import ModelTuner
from reclab.environments import LatentFactorBehavior
from env_defaults import TOPICS_STATIC, get_len_trial, LATENT_STATIC
from reclab.recommenders import EASE


In [4]:
# ====Step 4====
# S3 storage parameters
bucket_name = 'recsys-eval'
data_dir = 'master'
overwrite = True

# Experiment setup.
n_trials = 10
trial_seeds = [i for i in range(n_trials)]
len_trial = get_len_trial(LATENT_STATIC)

# Environment setup
environment_name = LATENT_STATIC['name']
env = LatentFactorBehavior(**LATENT_STATIC['params'], **LATENT_STATIC['optional_params'])

# Recommender setup
recommender_name = 'EASE'
recommender_class = EASE


In [5]:
# ====Step 5====
starting_data = get_env_dataset(env)


In [6]:
# ====Step 6====
# Recommender tuning setup
n_fold = 5
default_params = {}
tuner = ModelTuner(starting_data,
                   default_params,
                   recommender_class,
                   n_fold=n_fold,
                   verbose=True,
                   bucket_name=bucket_name,
                   data_dir=data_dir,
                   environment_name=environment_name,
                   recommender_name=recommender_name,
                   overwrite=overwrite)



In [7]:
# Tune the hyperparameter.
# Start with a coarse grid.
lams = np.linspace(10, 9000, 10).tolist()
tuner.evaluate_grid(lam=lams)


Evaluating: {'lam': 10.0}
Fold 1/5, mse=9.866568808100663, rmse=3.1411094868056835
Fold 2/5, mse=9.87308635996173, rmse=3.142146775687242
Fold 3/5, mse=9.818580275684642, rmse=3.1334613888932226
Fold 4/5, mse=9.959104903496797, rmse=3.1558049533354873
Fold 5/5, mse=9.994414882405385, rmse=3.161394452200703
Average MSE: 9.902351045929844
Evaluating: {'lam': 1008.8888888888889}
Fold 1/5, mse=9.388457817362532, rmse=3.0640590427344137
Fold 2/5, mse=9.39057483453168, rmse=3.0644044828533454
Fold 3/5, mse=9.333268861997281, rmse=3.055039911686471
Fold 4/5, mse=9.472508459156368, rmse=3.0777440535490226
Fold 5/5, mse=9.503839650900225, rmse=3.0828298121855875
Average MSE: 9.417729924789617
Evaluating: {'lam': 2007.7777777777778}
Fold 1/5, mse=9.290145344858814, rmse=3.0479739737830465
Fold 2/5, mse=9.29107459541297, rmse=3.0481264073874907
Fold 3/5, mse=9.235575260947504, rmse=3.0390089274214884
Fold 4/5, mse=9.373098986979844, rmse=3.061551728614077
Fold 5/5, mse=9.405489501114989, rmse=3.0

,lam,mse,average_mse
0,10.000000,"[9.866568808100663, 9.87308635996173, 9.818580...",9.902351
1,1008.888889,"[9.388457817362532, 9.39057483453168, 9.333268...",9.417730
2,2007.777778,"[9.290145344858814, 9.29107459541297, 9.235575...",9.319077
3,3006.666667,"[9.250313198890918, 9.2511309720304, 9.1965374...",9.279504
4,4005.555556,"[9.232999491197322, 9.233937062426895, 9.17987...",9.262519
5,5004.444444,"[9.226649217018126, 9.227760292942351, 9.17403...",9.256481
6,6003.333333,"[9.226328924741418, 9.22761872984934, 9.174118...",9.256441
7,7002.222222,"[9.229573619175579, 9.231032999104103, 9.17769...",9.259935
8,8001.111111,"[9.235019301684405, 9.236635326613987, 9.18341...",9.265605
9,9000.000000,"[9.241855142817423, 9.24361465662066, 9.190476...",9.272642


In [8]:
# Zeroing in on the best range.
lams = np.linspace(5000, 7000, 10).tolist()
tuner.evaluate_grid(lam=lams)

Evaluating: {'lam': 5000.0}
Fold 1/5, mse=9.226661676037857, rmse=3.0375420451473354
Fold 2/5, mse=9.227771950770935, rmse=3.037724798392859
Fold 3/5, mse=9.174042287692654, rmse=3.0288681529067345
Fold 4/5, mse=9.310003064163661, rmse=3.0512297625979694
Fold 5/5, mse=9.343979395909093, rmse=3.056792337714339
Average MSE: 9.256491674914841
Evaluating: {'lam': 5222.222222222223}
Fold 1/5, mse=9.226175199785404, rmse=3.037461966804754
Fold 2/5, mse=9.227325568882742, rmse=3.037651324441754
Fold 3/5, mse=9.173654306249897, rmse=3.028804104964515
Fold 4/5, mse=9.309574532764016, rmse=3.0511595390546224
Fold 5/5, mse=9.343622168766593, rmse=3.056733905456377
Average MSE: 9.256070355289731
Evaluating: {'lam': 5444.444444444444}
Fold 1/5, mse=9.225946340195847, rmse=3.0374242937389972
Fold 2/5, mse=9.227136741758141, rmse=3.0376202431769084
Fold 3/5, mse=9.173519165105498, rmse=3.0287817955583227
Fold 4/5, mse=9.309403935164536, rmse=3.051131582735254
Fold 5/5, mse=9.343519107923722, rmse=3.0

,lam,mse,average_mse
0,5000.000000,"[9.226661676037857, 9.227771950770935, 9.17404...",9.256492
1,5222.222222,"[9.226175199785404, 9.227325568882742, 9.17365...",9.256070
2,5444.444444,"[9.225946340195847, 9.227136741758141, 9.17351...",9.255905
3,5666.666667,"[9.225946582554064, 9.227176789073217, 9.17360...",9.255967
4,5888.888889,"[9.226151107580282, 9.227420765417946, 9.17389...",9.256232
5,6111.111111,"[9.226538216567906, 9.227846876758456, 9.17436...",9.256678
6,6333.333333,"[9.227088860887374, 9.228436003257833, 9.17499...",9.257286
7,6555.555556,"[9.227786254304391, 9.229171306414607, 9.17576...",9.258040
8,6777.777778,"[9.228615551523838, 9.230037903569684, 9.17666...",9.258924
9,7000.000000,"[9.229563580078363, 9.231022596632776, 9.17768...",9.259925


In [ ]:

# Set regularization to 5889.
lam = 5889


In [ ]:

# ====Step 7====
recommender = recommender_class(shrinkage=shrinkage, neighborhood_size=neighborhood_size)
for i, seed in enumerate(trial_seeds):
    run_env_experiment(
            [env],
            [recommender],
            [seed],
            len_trial,
            environment_names=[environment_name],
            recommender_names=[recommender_name],
            bucket_name=bucket_name,
            data_dir=data_dir,
            overwrite=overwrite)
